## U-Net: Convolutional networks for biomedical image segmentation (O. Ronneberger et al., 2015)

In [ ]:
from pathlib import Path
import pytorch_lightning as pl
from fastmri.data.subsample import create_mask_for_mask_type
from fastmri.data.transforms import UnetDataTransform
from fastmri.pl_modules import FastMriDataModule, UnetModule

import torch
from torch.nn import functional as F

import pytorch_lightning as pl

import fastmri
from fastmri import evaluate
from fastmri.pl_modules import MriModule
from fastmri.pl_modules.mri_module import DistributedMetricSum

## code below was adapted from the fastmri github in order to modify the UNET

## Training

### K-Space Mask for transforming the input data

In [ ]:
mask_types = [
    "random",
    "equispaced",
    "equispaced_fraction",
    "magic",
    "magic_fraction"
]
mask_type = mask_types[0]

In [ ]:
# Number of center lines to use in mask
center_fractions = [0.09]

In [ ]:
# acceleration rates to use for masks
accelerations = [4]

In [ ]:
mask = create_mask_for_mask_type(
    mask_type, center_fractions, accelerations
)
type(mask)

## Datasets

In [ ]:
# Data specific Parameters
data_path = Path('../data/')
test_path = Path('../data/singlecoil_test')
challenge = "singlecoil"
test_split = "test"

In [ ]:
# Fraction of slices in the dataset to use (train split only). 
# If not given all will be used. Cannot set together with volume_sample_rate.
sample_rate = 0.5
val_sample_rate = None
test_sample_rate = None
volume_sample_rate = None
val_volume_sample_rate = None
test_volume_sample_rate = None
use_dataset_cache_file = True
combine_train_val = False

# data loader arguments
batch_size = 1
num_workers = 0

### use random masks for train transform, fixed masks for val transform

In [ ]:
train_transform = UnetDataTransform(challenge, mask_func=mask, use_seed=False)
train_transform

In [ ]:
val_transform = UnetDataTransform(challenge, mask_func=mask)

In [ ]:
test_transform = UnetDataTransform(challenge)

In [ ]:
data_module = FastMriDataModule(
        data_path=data_path,
        challenge=challenge,
        train_transform=train_transform,
        val_transform=val_transform,
        test_transform=test_transform,
        test_split=test_split,
        test_path=test_path,
        sample_rate=sample_rate,
        distributed_sampler=None,
        num_workers=60,
        batch_size=80,
)
data_module.challenge

## Test Dataloaders

In [ ]:
train_dataloader = data_module.train_dataloader()
train_features = next(iter(train_dataloader))
train_features.image.shape, train_features.target.shape 

In [ ]:
val_dataloader = data_module.val_dataloader()
val_features = next(iter(val_dataloader))
val_features.image.shape, val_features.target.shape 

In [ ]:
test_dataloader = data_module.test_dataloader()
test_features = next(iter(test_dataloader))
test_features.image.shape, test_features.target.shape 

## UNet Model

In [ ]:
##############################
# UNet Model Hyperparameters #
##############################
in_chans=1          # number of input channels to U-Net
out_chans=1         # number of output chanenls to U-Net
chans=32            # number of top-level U-Net channels
num_pool_layers=4   # number of U-Net pooling layers
drop_prob=0.0       # dropout probability
lr=0.001            # RMSProp learning rate
lr_step_size=40     # epoch at which to decrease learning rate
lr_gamma=0.1        # extent to which to decrease learning rate
weight_decay=0.0    # weight decay regularization strength

In [ ]:
unet_model = UnetModule(
        in_chans=in_chans,
        out_chans=out_chans,
        chans=chans,
        num_pool_layers=num_pool_layers,
        drop_prob=drop_prob,
        lr=lr,
        lr_step_size=lr_step_size,
        lr_gamma=lr_gamma,
        weight_decay=weight_decay,
)
model = unet_model

## Trainer

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

trainer_config = dict(
    #replace_sampler_ddp=False,    # this is necessary for volume dispatch during val
    #strategy="ddp",               # what distributed version to use
    #seed=42,                      # random seed
    accelerator = "gpu",
    devices=1,                      # number of gpus to use
    deterministic=False,            # makes things slower, but deterministic
    default_root_dir='../logs',     # directory for logs and checkpoints
    max_epochs=50,                  # max number of epochs
    logger = TensorBoardLogger('../logs/', name='test')
)

trainer = pl.Trainer(**trainer_config)

## Run Training

In [ ]:
trainer.fit(model, datamodule=data_module)